## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-14-50-00 +0000,wapo,Working headline UPDATE,https://www.washingtonpost.com/nation/2025/09/...
1,2025-09-06-14-43-57 +0000,nyt,Lisbon Funicular Was Inspected Hours Before De...,https://www.nytimes.com/2025/09/05/world/europ...
2,2025-09-06-14-40-56 +0000,nypost,Brooklyn high school brazenly hangs Palestinia...,https://nypost.com/2025/09/06/us-news/brooklyn...
3,2025-09-06-14-35-49 +0000,nypost,"Serbian police unleash teargas, stun grenades ...",https://nypost.com/2025/09/06/world-news/anti-...
4,2025-09-06-14-19-54 +0000,nyt,"When It Comes to Spotting Fake Receipts, It’s ...",https://www.nytimes.com/2025/09/06/business/de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
206,trump,49
426,new,15
130,will,14
47,immigration,11
829,report,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,94
81,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,82
289,2025-09-05-15-43-16 +0000,nyt,Billionaire Crypto Investor Hits Out at Trump ...,https://www.nytimes.com/2025/09/05/business/tr...,82
76,2025-09-06-09-01-54 +0000,nyt,"George Mason’s president, Gregory Washington, ...",https://www.nytimes.com/2025/09/06/us/george-m...,81
86,2025-09-06-08-00-21 +0000,cbc,Trump's court losses are piling up. Here's why...,https://www.cbc.ca/news/world/trump-court-case...,78


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,94,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
126,54,2025-09-06-01-20-10 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
215,51,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
258,44,2025-09-05-17-33-52 +0000,latimes,CBS' 'Face the Nation' will no longer edit tap...,https://www.latimes.com/entertainment-arts/bus...
149,41,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
200,40,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
119,37,2025-09-06-02-17-09 +0000,nypost,Ex-FBI official tipped off Chinese firm doing ...,https://nypost.com/2025/09/05/us-news/ex-fbi-o...
97,35,2025-09-06-05-49-11 +0000,nypost,University of Kentucky cheerleader Laken Snell...,https://nypost.com/2025/09/06/us-news/universi...
14,34,2025-09-06-13-28-40 +0000,nypost,Manhattan DA demands sicko NYC super return $1...,https://nypost.com/2025/09/06/us-news/sicko-su...
290,31,2025-09-05-15-41-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
